In [1]:
from torch.utils.data import DataLoader, TensorDataset
import wandb

from m2_utilities.load_data import load_trajectories
from m2_utilities.preprocessor import Preprocessor
from m2_utilities.qwen import load_qwen, train
from m2_utilities.lora import apply_lora

%load_ext autoreload
%autoreload 2

In [2]:
model, tokenizer = load_qwen()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


In [3]:
LORA_RANK = 4
apply_lora(model, LORA_RANK)

### Preprocessing Trajectories to Tokens

In [10]:
# Load
trajectories = load_trajectories("data/lotka_volterra_data.h5")

# Train/validation
N_TRAIN = 700
N_VAL = 150

train = trajectories[:N_TRAIN]
val = trajectories[N_TRAIN: N_TRAIN + N_VAL]

# Preprocessing and chunking
DECIMALS = 2
preprocessor = Preprocessor(DECIMALS)
train_input_ids = preprocessor.encode(train, chunk=True)


In [ ]:
# Initialising data loaders
BATCH_SIZE = 4
train_dataset = TensorDataset(train_input_ids)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

### Training

In [ ]:
LR = 1e-5
config = {"Learning Rate": LR,
          "Batch Size": BATCH_SIZE, 
          "Decimals": DECIMALS,
          "Lora Rank": LORA_RANK}
wandb.init(project="M2 Coursework", config=config)
wandb.watch(model, log="all", log_freq=10)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: at2128 (at2128-university-of-cambridge) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [7]:
train(model, train_loader, val_loader, learning_rate=LR, eval_interval=50, wandb=wandb)

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Steps 0:   6%|▌         | 49/875 [01:12<20:25,  1.48s/it, loss=3.32] 


KeyboardInterrupt: 